# Project 4 K-Nearest Neighbors On Two Data Sets

## Part 1: Cleveland Data Set

In [ ]:
#Common imports for use in the data visualization
import math
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
import csv
import scipy.stats as stats

from sklearn.neighbors import NearestNeighbors
from sklearn.metrics import precision_recall_fscore_support

In [ ]:
clevelandDf = pd.read_csv('datasets/cleveland.csv')
clevelandDf
# Looking at the columns, The ones that seem the most helpful are
# Age, Sex, Chest pain type, resting electro... and resting blood pressure, serum cholestoral
 
interested_columns = ["age","sex","cp","restecg","trestbps","chol", "num"]
interestedDF = clevelandDf[interested_columns].copy()

diagnosesCount = clevelandDf["num"].value_counts()
print("Negative Tests:    ", diagnosesCount[0])
print("Level 1 Diagnosis: ", diagnosesCount[1])
print("Level 2 Diagnosis: ", diagnosesCount[2])
print("Level 3 Diagnosis: ", diagnosesCount[3])
print("Level 4 Diagnosis: ", diagnosesCount[4])
postiveCases = diagnosesCount[1] + diagnosesCount[2] + diagnosesCount[3] + diagnosesCount[4]
negativeCases = diagnosesCount[0]

#standardize the non binary data
# display(interestedDF)
interestedDF["age"] = (interestedDF["age"] - interestedDF["age"].mean()) / interestedDF["age"].std()
interestedDF["trestbps"] = (interestedDF["trestbps"] - interestedDF["trestbps"].mean()) / interestedDF["trestbps"].std()
interestedDF["chol"] = (interestedDF["chol"] - interestedDF["chol"].mean()) / interestedDF["chol"].std()
# interestedDF

interestedDF = interestedDF.sort_values(by="num")
interestedDF

nn = NearestNeighbors(n_neighbors=9, metric='euclidean', algorithm='auto')
nn.fit(interestedDF)

selectedTestCase = interestedDF.sample(1)##ranodomly select a test case
selectedTestCase


restOfDF =interestedDF.drop(selectedTestCase.index)
nn.fit(restOfDF)
distances, indices = nn.kneighbors(selectedTestCase)


distances, indices

## Part 2: Student Dropout Data Set

## [Good Datasets Here](https://archive.ics.uci.edu/)

In [ ]:
#pd.set_option('display.max_rows', None)
#pd.set_option('display.max_columns', None)

school_df = pd.read_csv('datasets/data.csv', sep=';')
interested_columns = ["Age at enrollment", "Tuition fees up to date", "Previous qualification (grade)", "Scholarship holder", "Target"]
school_interested = school_df[interested_columns].copy()

dropout_count = school_interested['Target'].value_counts().get('Dropout')
print("Number of dropouts:", dropout_count)
not_dropout_count = (school_interested['Target'] != 'dropout').sum()
print("Number of students who are not dropouts:", not_dropout_count)


# Standardize the non-binary data
school_interested['Age standardized'] = (school_interested['Age at enrollment'] - school_interested['Age at enrollment'].mean()) / (school_interested['Age at enrollment'].std())
school_interested['Tuition standardized'] = (school_interested['Tuition fees up to date'] - school_interested['Tuition fees up to date'].mean()) / (school_interested['Tuition fees up to date'].std())
school_interested['Grades standardized'] = (school_interested['Previous qualification (grade)'] - school_interested['Previous qualification (grade)'].mean()) / (school_interested['Previous qualification (grade)'].std())
school_interested['Scholarship standardized'] = (school_interested['Scholarship holder'] - school_interested['Scholarship holder'].mean()) / (school_interested['Scholarship holder'].std())

school_interested = school_interested.sort_values(by='Scholarship holder', ascending=False)

nn = NearestNeighbors(n_neighbors=9, metric='euclidean', algorithm='auto')

student = school_interested.sample(1)
studentX = student[['Age standardized', 'Grades standardized']].values[0]		# moved this block above
display(studentX)

# need to get rid of the patient itself
X = school_interested.drop(student.index)[['Age standardized', 'Grades standardized']].values

# Build index data structure under the hood for query performance
fit = nn.fit(X)

# Find k nearest neighbors
distances, indices = fit.kneighbors([studentX])
distances, indices

# Get the patients that are near the age
nbrs = school_interested.iloc[indices[0]]
display(nbrs)

# Print how many patients are sick and how many are healthy
in_school = nbrs[(nbrs['Target'] == "Enrolled") | (nbrs['Target'] == "Graduate")].count()
dropout = nbrs[nbrs['Target'] == "Dropout"].count()
#print('in school: {}\ndropout: {}'.format(in_school, dropout))

prediction = "isn't a dropout" if in_school['Target'] > dropout['Target'] else "is a dropout"
print("We predict this student is: " + prediction)

#display(school_interested)


In [ ]:
# Continue here

(p,r,f,s) = precision_recall_fscore_support(patientsy, y_pred, labels=[0,1])
print(f'precision={p}, recall={r}, f-score={f}, support={s}')